# Projet python 2024 : recherche de chemin optimal
## Xavier Juvigny

## I. Les algorithmes coopératifs

Le terme "*Intelligence en essaim*" a été introduit en 1989 par Beni et al. Cette classe d'algorithmes, souvent inspirés par le comportement des insectes sociaux, met en place une population d'agents simples interagissant et communicant indirectement avec leur environnement. Ces algorithmes constituent une classe d'algorithmes massivement parallèles pour résoudre une tâche donnée.

L'algorithme en essaim le plus connu est l'optimisation par colonies de fourmis (**ACO** : *Ants colonization optimisation*) pour les problèmes combinatoires. Dans ce projet, on revient à l'inspiration originale des algorithmes ACO où une population d'agents simples (qui peuvent être vus comme imitant le comportement de fourmis réelles) résout efficacement le problème de fourragement (chercher le chemin le plus cours de la fourmilière à une source de nourriture).

### I.1 Modèle simple d'ACO

Dans cette partie, nous allons décrire un modèle simple de colonies de fourmis qui permet de résoudre le problème de fourragement. Il a été montré pour cette algorithme qu'il converge de façon sur-linéaire par rapport au nombre de fourmis.

On utilisera un labyrinthe où il n'existe qu'un seul chemin sans rebroussement entre la fourmilière et la nourriture;

### I.2 Génération de l'environnement

#### I.2.a Création d'un labyrinthe

On crée un labyrinthe contenant $n_{i}\times n_{j}$ cases. Pour chaque case, on code les sorties de la manière suivante :

  - On assigne la valeur <font color=" #0000F0">1</font> à la direction <font color=" #F00000">Nord</font>;
  - On assigne la valeur <font color=" #0000F0">2</font> à la direction <font color=" #F00000">Est</font>;
  - On assigne la valeur <font color=" #0000F0">4</font> à la direction <font color=" #F00000">Sud</font>;
  - On assigne la valeur <font color=" #0000F0">8</font> à la direction <font color=" #F00000">Ouest</font>;
  - Pour une case donnée, on lui assigne la somme des valeurs correspondantes aux sorties possible à partir de cette cellule.

<p style="background-color: #808080;text-align: justify">
<b><u>Exemple</u></b> : Une case du labyrinthe ayant une sortie <i>Nord</i> et une sortie <i>Ouest</i>, la valeur assignée à cette case sera égale à <i>8 (Ouest) + 1 (Nord) = 9</i>.

<font color=" #8000FF"><u>Algorithme de création du labyrinthe</u></font>:

- Initialiser un tableau `rooms` de taille $(n_{i},n_{j})$ initialisé à zéro décrivant les sorties du labyrinthe;
- Initialiser une liste `historic` (vide) des cases déjà visitées mais qu'on pourra "revisiter" pour rajouter des sorties;
- Initialiser un tableau booléen `visited` indiquant si une case a déjà été visitée (`True`) ou non (`False`);
- Rajouter la case `(0,0)` à la liste `historic`;
- Tant que `historic` n'est pas vide:
    - Dépiler la case `c` de `historic`;
    - Créer une liste `V` contenant les cases adjacentes à `c` non visitées;
    - Si `V` n'est pas vide:
      - Tirer au hasard une case `v`$\in$`V`;
      - Rajouter à `c` la sortie connectant `c` à `v`;
      - Empiler `v` dans `historic`
      - Assigner `True` pour la case `c` dans `visited`
    - sinon on dépile `c` de `historic`.

#### I.2.b Affichage du labyrinthe

Dans un premier temps, nous allons afficher un labyrinthe généré avec l'algorithme pour vérifier si le labyrinthe a bien été généré.

Pour cela, nous allons utiliser le *package* `pygame` et le fichier `cases.png` qui permettra de dessiner chaque case du labyrinthe.

La première chose, c'est d'importer `pygame` et d'initialiser son contexte :

In [ ]:
import pygame
pygame.init()

Il faut ensuite définir la fenêtre graphique sur laquelle on affichera les différents sprites. On devra spécifier la résolution de l'écran :

In [4]:
screen = pygame.display.set_mode((320,200))

Nous allons ensuite charger l'image des différentes configurations de cases possibles en prenant en compte la transparence :

In [6]:
cases_img = pygame.image.load("cases.png").convert_alpha()

On peut déjà afficher l'image sur l'écran (image contenant toutes les configurations possibles de cases). On remplit d'abord l'écran avec un fond blanc afin de pouvoir voir l'image (les murs sont en noir !)

In [ ]:
screen.fill((255,255,255))
screen.blit(cases_img, (0,0))

Vous voyez qu'on ne voit rien apparaître à l'écran. En fait, il faut mettre à jour l'écran pour que l'affichage soit effectif (technique de double buffering)
**Note** : L'affichage ne semble pas marcher sous jupyter notebook. Il faudra donc recopier les bouts de code dans un script python classique !

In [ ]:
pygame.display.update()

On veut maintenant subdiviser l'image en huit images, chacune représentant une case avec les sorties adéquates. On utilise pour cela 
```subimg = pygame.Surface.subsurface(image, x, y, width, height)```

In [ ]:
cases_spr = []
for i in range(0, 128, 8):
    cases_spr.append(pygame.Surface.subsurface(cases_img, i, 0, 8, 8))

L'indices des sprites correspond à la représentation de chaque valeur pouvant être prise par une cellule.

## Description du modèle

Soit un labyrinthe 2D, dont chaque case comporte une valeur donnant les sorties possibles (voir plus haut). 

On indique la case où se trouve la fourmilière (par exemple (0,0)) et la case où se trouve la nourriture (par exemple ($n_{i}-1,n_{j}-1$)).

On considère un ensemble de $m$ fourmis artificielles. Chaque fourmi possèdera les données suivantes :

  - La position (dans quelle case se trouve la fourmi);
  - Son orientation (dans quelle direction pointe-t'elle sa tête ?);
  - Son état : "*non chargée*" ou "*chargée*" (de nourriture);
  - L'historique des cases par lesquelles la fourmi est passée (pouvant contenir plusieurs fois la même case)
  - L'âge maximale que peut atteindre la fourmi;
  - L'âge de la fourmi.

L'orientation de la fourmi permet d'empêcher au coup suivant que la fourmi fasse demi-tour si elle peut l'éviter. Les valeurs de directions sont :

 - ```DIR_NONE = -1``` : à l'initialisation pas de direction privilégiée
 - ```DIR_NORTH = 0``` : La fourmi est dirigée vers le Nord
 - ```DIR_EAST = 1``` : La fourmi est dirigée vers l'Est'
 - ```DIR_WEST = 2``` : La fourmi est dirigée vers l'Ouest'
 - ```DIR_SOUTH = 3``` : La fourmi est dirigée vers le Sud.

 Les numéros associés à chaque direction ```d``` ont été choisi de sorte que pour obtenir la direction opposée, on n'a qu'à calculer ```3-d```

L'historique permet à la fourmi, dès qu'elle se charge de nourriture, de refaire le chemin inverse pour rapporter la nourriture à la fourmilière.

L'âge maximale de la fourmi permet de savoir à quel âge elle va mourir. Faire mourir une fourmi permet d'éliminer les fourmis qui se sont "perdues" dans le labyrinthe. Quand une fourmi meurt, on la repositionne dans la fourmilière et on remet son âge à zéro.

On crée aussi une carte de phéromone qui pour chaque case du maillage, indique un taux de phéromone qui servira aux fourmis à se guider vers la nourriture.

## Comportement du phéromone

A l'initiation, la carte de phéromone est nulle partour sauf à l'endroit où se trouve la nourriture où la valeur vaut un.

A chaque pas de temps, sauf à l'endroit où se trouve la nourriture (où la valeur vaut constamment un), les phéromones s'évaporent selon un taux $\beta < 1$ : $p = \beta.p$. Plus le paramètre $\beta$ s'approche de 0, plus les phéromones laissés par les fourmis vont disparaître rapidement, plus la valeur $\beta$ se rapprochera de un, plus les phéromones seront persistants.

  - Une évaporation assez forte permet d'éliminer rapidement les phéromones se trouvant dans des impasses où est passée une fourmi chargée qui refait son chemin inverse, mais risque aussi de trop rapidement évaporer les phéromones menant directement à la nourriture !
  - A l'inverse, une évaporation pas assez forte permet certe de retrouver la nourriture mais risque également de conserver les chemins qui amènent à des impasses.

La valeur par défaut de $\beta$ est $0.999$.

## Comportement des fourmis

Les comportements des fourmis dépendent si elles sont chargés ou non.

### Comportement d'une fourmi non chargée

On se sert pour cela d'un coefficient d'exploration $\varepsilon$ (valant par défaut zéro).

On ne considère comme cases adjacente $\mathcal{V}$ à la fourmi que les cases adjacentes ayant une sortie commune avec la case où se  trouve la fourmi. 

Si toutes les cases $c\in\mathcal{V}$ ont une valeur nulle de phéromone, la fourmi prend une sortie au hasard en essayant si possible de ne pas faire demi-tour.

Si une case ou plus $c\in\mathcal{V}$ possède une valeur non nulle de phéromone, on tire au hasard un nombre entre 0 et 1. Si ce nombre est inférieur à $\varepsilon$, alors la fourmi se comporte comme si les phéromones autour d'elle étaient nuls. Sinon, elle prendra automatiquement la case où le phéromone a la valeur la plus forte (et qui évite si possible de faire demi-tour).

Si la fourmi arrive sur la case possédant la nourriture, elle devient une fourmi chargée, sinon on augmente son âge de un, et on vérifie si elle doit mourir où non.

Si elle meurt, on la repositionne à l'endroit de la fourmilière et on remet son âge à zéro.

### Comportement d'une fourmi chargée

Pour une fourmi chargée, on va utiliser le paramètre 
$\alpha$ donnant le taux de copie de phéromone, le complément $1-\alpha$ donnant le taux de diffusion.

Ainsi une fourmi chargée va à chaque pas de temps dépiler son historique pour repasser par toutes les cases qu'elle a parcouru depuis la fourmilière. 

A chaque pas de temps, à partir de sa nouvelle position $h$, on regarde également les phéromones accessibles (correspondant aux cellules $c\in\mathcal{V}$, et on met à jour le phéromone $p(h)$ où se trouve la fourmi avec la formule suivant :

$$
p(h) = \alpha.\max_{c\in\mathcal{V}}(p(c)) + \frac{1-\alpha}{4}\sum_{c\in\mathcal{V}}p(c)
$$

Si la fourmi arrive à la fourmillière, on incrémente le compteur de nourriture de un, on remet la fourmi à l'état non chargée et on remet son âge à zéro.

**Remarque** : Une fourmi chargée ne veillit pas tant qu'elle est chargée !

## L'affichage

L'affichage consiste à afficher tout d'abord les phéromones en coloriant chaque case en fonction du taux de phéromone.

**Conseil** : Il est plus intéressant de colorier avec une certaine composante couleur les cases dont la valeur a dépassé un certain seuil (par exemple plus de $10^{-8}$) et rajouter une autre composante proportionnelle au taux de phéromone contenu par la cellule.

On affiche ensuite avec la transparence le labyrinthe puis chaque fourmi en fonction de sa position et sa direction.

## Parallélisation du code

Enfin, on essayera de paralléliser le code en 
  - Séparant affichage (sur le proc 0) et gestion des fourmiS/phéromones (sur le proc 1)
  - Puis en partitionnant les fourmis entre les processus dont le rang est non nul (le zéro continuant à gérer l'affichage)
  - Réfléchir comment on pourrait partitionner le labyrinthe pour gérer les fourmis en parallèle sur un labyrinthe distribué entre les processus

On rendra un document de préférence pdf pour les diverses réflexions induits par le projet, à savoir :
  - Quelles sont les parties du code parallélisable lorsqu'on partitionne uniquement les fourmis
  - Quels gains on a obtenu (speed-up) ?
  - Décrire vos réflexions de comment vous voyez la mise en oeuvre du code en parallèle si on partionne en plus le labyrinthe entre les divers procesus.